# Formalização da Regressão Logística

## 1) Descrição

### A regressão logística analisa dados distribuídos binomialmente da forma $Y_i \ \sim  B(p_i,n_i),\text{ for }i = 1, \dots , m,$ onde os números de `Distribuição de Bernoulli` $n_i$ são conhecidos e as probabilidades de êxito $p_{i}$ são desconhecidas. Um exemplo desta distribuição é a percentagem de sementes $p_{i}$ que germinam depois de $n_i$ serem plantadas.

### O modelo é então obtido na base de que cada ensaio (valor de $i$) e o conjunto de variáveis explicativas/independentes possa informar acerca da probabilidade final. Estas variáveis explicativas podem-se ver como um vector $X_i$ $k$-dimensional e o modelo toma então a forma:

$p_i = \operatorname{E}\left(\left.\frac{Y_i}{n_{i}}\right|X_i \right). \,\!$

### Os logits das probabilidades binomiais desconhecidas ("i.e.", os logaritmos dos `odds`) são modelados como uma função linear dos $X_i$. $\operatorname{logit}(p_i)=\ln\left(\frac{p_i}{1-p_i}\right) = \beta_0 + \beta_1 x_{1,i} + \cdots + \beta_k x_{k,i}.$ Note-se que um elemento particular de $X_i$ pode ser ajustado a 1 para todo o ''i'' obtendo-se um intercepto no modelo. Os parâmetros desconhecidos $β_j$ são habitualmente estimados através de `máxima verossimilhança`.

### A interpretação dos valores estimados do parâmetro $β_j$ é similar aos efeitos aditivos em log `odds ratio` para uma unidade de mudança na $j$-ésima variável explicativa. No caso de uma variável explicativa dicotómica, por exemplo o género, $e^\beta$ é o estimador de odds ratio de ter o resultado para, por exemplo, homens comparados com mulheres.

### O modelo tem uma formulação equivalente dada por $p_i = \frac{1}{1+e^{-(\beta_0 + \beta_1 x_{1,i} + \cdots + \beta_k x_{k,i})}}. \,\!$ 

### Esta forma funcional é habitualmente identificada como um perceptron (ou "perceptrão" em português europeu) de uma camada simples ou `rede neural artificial` de uma só camada. Uma rede neuronal de uma só camada calcula uma saída contínua em vez de uma "função por troços". A derivada de $p_i$ em relação a $X = x_1...x_k$ é calculada na forma geral:

$y = \frac{1}{1+e^{-f(X)}}$, 

### onde $f(X)$ é uma `função analítica` em $X$. Com esta escolha, a rede de camada simples é idêntica ao modelo de regressão logística. Esta função tem uma derivada contínua, a qual permite ser usada na propagação para trás. Esta função também é preferida pois a sua derivada é facilmente calculável:

$y' = y(1-y)\frac{\mathrm{d}f}{\mathrm{d}X}\,\!$


# Caso de Uso: `Spam de E-mail`

In [1]:
#install.packages("kernlab", dependencies=TRUE)
library(kernlab)
#install.packages("caret", dependencies=TRUE)
library(caret)
#install.packages("lattice")
library(lattice)
#install.packages("ggplot2")
library(ggplot2)
#install.packages("ROCR") # Download package here: https://rocr.bioinf.mpi-sb.mpg.de
#library(ROCR)

Loading required package: lattice
Loading required package: ggplot2

Attaching package: 'ggplot2'

The following object is masked from 'package:kernlab':

    alpha

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done


In [2]:
data(spam)
print(dim(spam))

[1] 4601   58


In [3]:
head(spam)

make,address,all,num3d,our,over,remove,internet,order,mail,...,charSemicolon,charRoundbracket,charSquarebracket,charExclamation,charDollar,charHash,capitalAve,capitalLong,capitalTotal,type
0.00,0.64,0.64,0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.00,0.000,0,0.778,0.000,0.000,3.756,61,278,spam
0.21,0.28,0.50,0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.00,0.132,0,0.372,0.180,0.048,5.114,101,1028,spam
0.06,0.00,0.71,0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.01,0.143,0,0.276,0.184,0.010,9.821,485,2259,spam
0.00,0.00,0.00,0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.137,0,0.137,0.000,0.000,3.537,40,191,spam
0.00,0.00,0.00,0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.135,0,0.135,0.000,0.000,3.537,40,191,spam
0.00,0.00,0.00,0,1.85,0.00,0.00,1.85,0.00,0.00,...,0.00,0.223,0,0.000,0.000,0.000,3.000,15,54,spam


In [4]:
summary(spam)

      make           address            all             num3d         
 Min.   :0.0000   Min.   : 0.000   Min.   :0.0000   Min.   : 0.00000  
 1st Qu.:0.0000   1st Qu.: 0.000   1st Qu.:0.0000   1st Qu.: 0.00000  
 Median :0.0000   Median : 0.000   Median :0.0000   Median : 0.00000  
 Mean   :0.1046   Mean   : 0.213   Mean   :0.2807   Mean   : 0.06542  
 3rd Qu.:0.0000   3rd Qu.: 0.000   3rd Qu.:0.4200   3rd Qu.: 0.00000  
 Max.   :4.5400   Max.   :14.280   Max.   :5.1000   Max.   :42.81000  
      our               over            remove          internet      
 Min.   : 0.0000   Min.   :0.0000   Min.   :0.0000   Min.   : 0.0000  
 1st Qu.: 0.0000   1st Qu.:0.0000   1st Qu.:0.0000   1st Qu.: 0.0000  
 Median : 0.0000   Median :0.0000   Median :0.0000   Median : 0.0000  
 Mean   : 0.3122   Mean   :0.0959   Mean   :0.1142   Mean   : 0.1053  
 3rd Qu.: 0.3800   3rd Qu.:0.0000   3rd Qu.:0.0000   3rd Qu.: 0.0000  
 Max.   :10.0000   Max.   :5.8800   Max.   :7.2700   Max.   :11.1100  
     o

### O Dataset apresenta *58 colunas* e *4.601 linhas*

### A base tem diversas características de e-mails, como `quantidade de letras maiúsculas`, `quantidade de ponto e vírgula`, dentre outros e, por fim, a coluna `type`, marcando se o e-mail é *_$um spam ou não_$*. 

### Ou seja, temos um histórico de e-mails, as informações deles e se o e-mail é um spam ou não. Com isso, podemos tentar construir um modelo que seja capaz de prever se um e-mail é um spam ou não.

### Em primeiro lugar, precisamos separar a base *treino* e *teste*. Para facilitar o trabalho, vamos carregar o `pacote caret` que possui várias funções de modelagem. Em seguida, utilizamos a *função createDataPartition* para separar a base de forma aleatória na proporção: `75%|25%`. Com base na separação feita, criamos a base *treino* e o que não estiver nela será a base teste:m

In [5]:
indice.treino <- createDataPartition(y=spam$type, p=0.75, list=FALSE)
treino = spam[indice.treino, ]
teste = spam[-indice.treino, ]

In [6]:
sapply(treino, function(x) sum(is.na(x)))

make           address               all             num3d 
                0                 0                 0                 0 
              our              over            remove          internet 
                0                 0                 0                 0 
            order              mail           receive              will 
                0                 0                 0                 0 
           people            report         addresses              free 
                0                 0                 0                 0 
         business             email               you            credit 
                0                 0                 0                 0 
             your              font            num000             money 
                0                 0                 0                 0 
               hp               hpl            george            num650 
                0                 0                 0                 0 
              lab              labs            telnet            num857 
                0                 0                 0                 0 
             data            num415             num85        technology 
                0                 0                 0                 0 
          num1999             parts                pm            direct 
                0                 0                 0                 0 
               cs           meeting          original           project 
                0                 0                 0                 0 
               re               edu             table        conference 
                0                 0                 0                 0 
    charSemicolon  charRoundbracket charSquarebracket   charExclamation 
                0                 0                 0                 0 
       charDollar          charHash        capitalAve       capitalLong 
                0                 0                 0                 0 
     capitalTotal              type 
                0                 0

In [7]:
modelo = glm(type ~ our+over+remove, data=treino, family=binomial)
summary(modelo)

Warning message:
"glm.fit: fitted probabilities numerically 0 or 1 occurred"


Call:
glm(formula = type ~ our + over + remove, family = binomial, 
    data = treino)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-5.6425  -0.7363  -0.7363   0.8773   1.6957  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept) -1.16667    0.04826 -24.177   <2e-16 ***
our          0.60880    0.07197   8.459   <2e-16 ***
over         2.49746    0.21766  11.474   <2e-16 ***
remove       5.56538    0.45339  12.275   <2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 4628.1  on 3450  degrees of freedom
Residual deviance: 3658.0  on 3447  degrees of freedom
AIC: 3666

Number of Fisher Scoring iterations: 7


In [8]:
summary(modelo)$coefficients

,Estimate,Std. Error,z value,Pr(>|z|)
(Intercept),-1.1666699,0.04825507,-24.177147,3.870433e-129
our,0.6087972,0.07197131,8.458887,2.699418e-17
over,2.4974555,0.21766400,11.473903,1.784271e-30
remove,5.5653844,0.45339214,12.274991,1.234053e-34


In [9]:
odd.ratio = exp(coef(modelo))
odd.ratio

(Intercept)         our        over      remove 
  0.3114022   1.8382190  12.1515353 261.2256103

In [10]:
pred.Teste = predict(modelo,teste, type = "response")
Teste_v2 = cbind(teste,pred.Teste)
Teste_v2

,make,address,all,num3d,our,over,remove,internet,order,mail,...,charRoundbracket,charSquarebracket,charExclamation,charDollar,charHash,capitalAve,capitalLong,capitalTotal,type,pred.Teste
4,0.00,0.00,0.00,0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.137,0.000,0.137,0.000,0.000,3.537,40,191,spam,0.7195320
7,0.00,0.00,0.00,0,1.92,0.00,0.00,0.00,0.00,0.64,...,0.054,0.000,0.164,0.054,0.000,1.671,4,112,spam,0.5005552
9,0.15,0.00,0.46,0,0.61,0.00,0.30,0.00,0.92,0.76,...,0.271,0.000,0.181,0.203,0.022,9.744,445,1257,spam,0.7056421
15,0.00,0.00,1.42,0,0.71,0.35,0.00,0.35,0.00,0.71,...,0.102,0.000,0.357,0.000,0.000,1.971,24,205,spam,0.5348647
22,0.05,0.07,0.10,0,0.76,0.05,0.15,0.02,0.55,0.00,...,0.101,0.016,0.250,0.046,0.059,2.569,66,2259,spam,0.5635781
24,0.00,0.00,0.00,0,1.16,0.00,0.00,0.00,0.00,0.00,...,0.133,0.000,0.667,0.000,0.000,1.131,5,69,spam,0.3868755
25,0.00,0.00,0.00,0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.196,0.000,0.392,0.196,0.000,5.466,22,82,spam,0.2374574
26,0.05,0.07,0.10,0,0.76,0.05,0.15,0.02,0.55,0.00,...,0.101,0.016,0.250,0.046,0.059,2.565,66,2258,spam,0.5635781
27,0.00,0.00,0.00,0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.196,0.000,0.392,0.196,0.000,5.466,22,82,spam,0.2374574
40,0.00,0.41,1.66,0,0.41,0.00,0.00,0.00,0.00,0.00,...,0.068,0.000,0.750,0.000,0.000,3.851,121,285,spam,0.2855567


In [11]:
#install.packages("ROCR", dependencies = TRUE)
#library(ROCR)
pred.val = prediction(pred.Teste ,Teste_v2$type)

ERROR: Error in prediction(pred.Teste, Teste_v2$type): could not find function "prediction"


In [ ]:
# Plota curva ROC
performance = performance(pred.val, "tpr", "fpr")
plot(performance, col = "blue", lwd = 5)

In [ ]:
# calculo da auc (area under the curve)
auc = performance(pred.val,"auc")
auc

In [ ]:
#Calculo Estatística KS
ks <- max(attr(performance, "y.values")[[1]] - (attr(performance, "x.values")[[1]]))
ks